In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install  transformers datasets accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import transformers
print(transformers.__version__)

4.53.3


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset


In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/AnonymousSub/MedQuAD_47441_Question_Answer_Pairs/data/train-00000-of-00001-4401d00b2bdd1863.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(df.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47441 entries, 0 to 47440
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Questions  47441 non-null  object
 1   Answers    16407 non-null  object
dtypes: object(2)
memory usage: 741.4+ KB
None


In [ ]:
# Normalize weird phrasing

# Step 1: Select and rename
df["prompt"] = df["Questions"].str.strip()
df["response"] = df["Answers"].str.strip()

df["prompt"] = df["prompt"].str.replace(r"\(are\)", "is", regex=True)
# ✅ Step 2: Drop real NaN first
df = df.dropna(subset=["prompt", "response"])

# ✅ Step 3: Drop if still any empty or 'None' as string
df = df[(df["prompt"].str.lower() != "none") & (df["response"].str.lower() != "none")]
df = df[(df["prompt"].str.strip() != "") & (df["response"].str.strip() != "")]

# ✅ Step 4: Now convert to string for safety
df["prompt"] = df["prompt"].astype(str)
df["response"] = df["response"].astype(str)

# ✅ Final format
final_df = df[["prompt", "response"]]

# ✅ Confirm
print(final_df.sample(3))
print(f"\n✅ Cleaned dataset ready with {len(final_df)} samples.")




                                                 prompt  \
169   What is is What I need to know about Kidney St...   
219         How to diagnose Kidney Stones in Children ?   
4090  What are the genetic changes related to genito...   

                                               response  
169   A kidney* stone is a solid piece of material t...  
219   The process of diagnosing any illness begins w...  
4090  Genitopatellar syndrome is caused by mutations...  

✅ Cleaned dataset ready with 16407 samples.


In [ ]:
from datasets import Dataset
dff = Dataset.from_pandas(final_df)
print(dff[0])

{'prompt': 'What is is Hepatitis B: What Asian and Pacific Islander Americans Need to Know ?', 'response': 'Hepatitis B is a liver disease spread through contact with blood, semen, or other body fluids from a person infected with the hepatitis B virus. The disease is most commonly spread from an infected mother to her infant at birth. Hepatitis B is also spread through sex, wound-to-wound contact, and contact with items that may have blood on them, such as shaving razors, toothbrushes, syringes, and tattoo and body piercing needles.\n                \nHepatitis B is not spread through casual contact such as shaking hands or hugging; nor is it spread by sharing food or beverages, by sneezing and coughing, or through breastfeeding.', '__index_level_0__': 0}


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    # Combine prompt + response for each example in batch
    full_texts = [p + " " + r for p, r in zip(examples["prompt"], examples["response"])]

    # Tokenize all at once (batch)
    tokenized = tokenizer(
        full_texts,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # GPT-style: labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.resize_token_embeddings(len(tokenizer))  # Token embeddings adjust kiye
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-medical-finetuned",       # Output directory
    eval_strategy="no",                      # No evaluation
    per_device_train_batch_size=4,              # Batch size per device
    num_train_epochs=1,                         # Number of training epochs
    save_steps=500,                             # Save model every 500 steps
    # eval_steps=500,                           # Commented, kyunki eval_strategy="no"
    logging_steps=100,                          # Log every 100 steps
    warmup_steps=100,                           # Learning rate warmup
    weight_decay=0.01,                          # Regularization
    save_total_limit=2,                         # Max saved checkpoints
    logging_dir="./logs",                       # Logging directory
    fp16=True,                                  # Use mixed precision (if GPU supports)
    report_to="none"                            # Disable external reporting
)

tokenized_data = dff.map(tokenize_function, batched=True)





Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args= training_args,
    train_dataset=tokenized_data,
    tokenizer = tokenizer
)

/tmp/ipython-input-13-3066711429.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.502300
200,1.039300
300,0.956100
400,0.839200
500,0.948200
600,0.879300
700,0.884600
800,0.868700
900,0.825800
1000,0.832200


TrainOutput(global_step=4102, training_loss=0.8525387892950924, metrics={'train_runtime': 1594.8226, 'train_samples_per_second': 10.288, 'train_steps_per_second': 2.572, 'total_flos': 4287018369024000.0, 'train_loss': 0.8525387892950924, 'epoch': 1.0})

In [ ]:
model.save_pretrained('gpt2-medical-finetuned1')

In [ ]:
tokenizer.save_pretrained("gpt2-medical-finetuned1")

('gpt2-medical-finetuned1/tokenizer_config.json',
 'gpt2-medical-finetuned1/special_tokens_map.json',
 'gpt2-medical-finetuned1/vocab.json',
 'gpt2-medical-finetuned1/merges.txt',
 'gpt2-medical-finetuned1/added_tokens.json',
 'gpt2-medical-finetuned1/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load fine-tuned model
medical_bot = pipeline(
    "text-generation",
    model="gpt2-medical-finetuned1",
    tokenizer="gpt2-medical-finetuned1",
    pad_token_id=50256  # GPT-2 ka eos_token_id
)

# Test with prompt
prompt = "What is the best treatment for diabetes?"
response = medical_bot(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)[0]['generated_text']
print("🔍 Model Response:\n", response)

Device set to use cuda:0


🔍 Model Response:
 What is the best treatment for diabetes? The goals of treatment for diabetes include
                
- reducing the risk of developing diabetes  - increasing the number of healthy blood vessels  - improving the quality of care
                
Diabetes treatment usually starts in the first few years of life. The goal is to improve the quality of care by reducing the risk of developing diabetes.
   


**🧠 1. Load Fine-Tuned Model**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2-medical-finetuned1')
model = AutoModelForCausalLM.from_pretrained('gpt2-medical-finetuned1')

# Test function
def test_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with prompt
prompt = "What is the best treatment for diabetes?"
print("🔍 Model Response:\n", test_model(prompt))

🔍 Model Response:
 What is the best treatment for diabetes? A treatment that focuses on the kidneys is not a cure. There is no cure for diabetes. There are different types of medicines that can help people manage the symptoms of diabetes. These medicines are used by many people to manage the symptoms of diabetes. Some medicines are used by people who have only mild symptoms. Others may be used by people who are at risk for diabetes.


**2. Define Test Function**



**3. Run Test (Give Prompt!)**

In [ ]:
test_prompts = [
    "What is the first sign of Hepatitis B?",
    "How can I manage high blood pressure?",
    "Symptoms of type 2 diabetes?",
    "Is Hepatitis B contagious?"
]

for p in test_prompts:
    print(f"\n🧪 Prompt: {p}")
    print("🔍 Response:", test_model(p))



🧪 Prompt: What is the first sign of Hepatitis B?
🔍 Response: What is the first sign of Hepatitis B? Signs and symptoms of hepatitis B may include fever, muscle aches, and fatigue. Symptoms may also include vomiting, diarrhea, and abdominal pain. Symptoms may include constipation and abdominal pain. Hepatitis B has been described as a complication of a blood clot, but the underlying cause is unknown.

🧪 Prompt: How can I manage high blood pressure?
🔍 Response: How can I manage high blood pressure?
               
Overview
                
High blood pressure (HBP) can be treated with medications.
                
Medicines
                 
Medications are medicines that help people with high blood pressure avoid

🧪 Prompt: Symptoms of type 2 diabetes?
🔍 Response: Symptoms of type 2 diabetes? Type 2 diabetes can be diagnosed by having a blood test. The blood test is used to find out whether you have type 2 diabetes. It is also used to diagnose type 3 diabetes.
                 
Blood t

In [ ]:
!zip -r model.zip gpt2-medical-finetuned1

  adding: gpt2-medical-finetuned1/ (stored 0%)
  adding: gpt2-medical-finetuned1/model.safetensors

In [ ]:
{
  "metadata": {
    "widgets": {
      "state": {}
    }
  }
}

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


import zipfile
import os

zip_path = "/content/drive/MyDrive/finetuned models/model.zip"
extract_path = "/content/model"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check contents
os.listdir(extract_path)

['gpt2-medical-finetuned1']

In [7]:
from huggingface_hub import upload_folder

# Update repo_id yahan:
repo_id = "samirk10/fine-tune"  # replace with your actual repo ID
local_model_path = "model/gpt2-medical-finetuned1"  # tumhara fine-tuned model folder

upload_folder(
    repo_id=repo_id,
    folder_path=local_model_path,
    path_in_repo=".",  # root pe upload
    commit_message="Pushing fine-tuned GPT-2 medical model 🚀"
)


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/samirk10/fine-tune/commit/b3f683aed0c2dc9415b39de66459026dd68c0b60', commit_message='Pushing fine-tuned GPT-2 medical model 🚀', commit_description='', oid='b3f683aed0c2dc9415b39de66459026dd68c0b60', pr_url=None, repo_url=RepoUrl('https://huggingface.co/samirk10/fine-tune', endpoint='https://huggingface.co', repo_type='model', repo_id='samirk10/fine-tune'), pr_revision=None, pr_num=None)